# Binary Tempotron
An implementation of the binary tempotron in Julia. 
For further details see [Gütig, R., & Sompolinsky, H. (2006). The tempotron: a neuron that learns spike timing–based decisions. Nature neuroscience, 9(3), 420.](https://www.nature.com/articles/nn1643).

## Imports

In [6]:
using Tempotrons
using Tempotrons.InputGen
using Tempotrons.Optimizers
using Plots
using Plots.PlotMeasures;

## Set parameters

In [7]:
N = 10
T = 500
dt = 1
t = collect(0:dt:T)
λ = 1e-4
opt = SGD(λ, momentum = 0.99)
ν = 3
n_samples = 10
n_epochs = 2000
tmp = Tempotron(N);

## Generate input samples

In [8]:
base_samples = [poisson_spikes_input(N, ν = ν, T = T) for j = 1:2]
samples = [(x = spikes_jitter(base_samples[2(j - 1) ÷ n_samples + 1], σ = 5),
            y = Bool(2(j - 1) ÷ n_samples)) for j = 1:n_samples];

## Training

In [9]:
# Get the tempotron's output before training
out_b = [tmp(s.x, t = t) for s ∈ samples]

# Train the tempotron
@time train!(tmp, samples, epochs = n_epochs, optimizer = opt)

# Get the tempotron's output after training
out_a = [tmp(s.x, t = t) for s ∈ samples];

  0.897409 seconds (13.19 M allocations: 584.281 MiB, 17.54% gc time)


## Plots

In [10]:
plotlyjs(size = (700, 1500))
cols = collect(1:2)#palette(:rainbow, 2)

inp_plots = map(samples) do s
    return plot(s.x, color = cols[1 + s.y], markersize = sqrt(5))
end
train_plots = map(zip(samples, out_b, out_a)) do (s, ob, oa)
    p = plot(tmp, t, oa.V, color = cols[1 + s.y])
    plot!(tmp, t, ob.V, color = cols[1 + s.y], linestyle = :dash)
    txt, clr = Tempotrons.get_progress_annotations(length(oa.spikes) > 0,
                                                   N_b = length(ob.spikes) > 0,
                                                   N_t = s.y)
    annotate!(xlims(p)[1], ylims(p)[2], text(txt, 10, :left, :bottom, clr))
    return p
end
ip = plot(inp_plots..., layout = (length(inp_plots), 1), link = :all)
tp = plot(train_plots..., layout = (length(train_plots), 1), link = :all)
p = plot(ip, tp, layout = (1, 2), left_margin = 8mm, bottom_margin = 15mm)

<!DOCTYPE html>
 
 
 Plots.jl